# Notebook 1: Extract Z-scored LFP

### Overview
This Jupyter notebook focuses on extracting local field potential (LFP) traces from Spikegadgets `.rec` files, specifically for neuroscience research related to social competition trials. The notebook includes procedures for preprocessing and synchronizing raw electrophysiology data with corresponding video data and computes various metrics, including Z-scored LFPs.

### Inputs & Data Sources
- **Electrophysiology and LFP Parameters**: Constants like `EPHYS_SAMPLING_RATE`, `LFP_SAMPLING_RATE`, `TRIAL_DURATION`, etc., define basic parameters for LFP data processing.
- **Recording Information**: Stream IDs (`ECU_STREAM_ID`, `TRODES_STREAM_ID`), recording extension (`RECORDING_EXTENSION`), and paths to recording directories (`ALL_SESSION_DIR`).
- **DataFrames for Mapping and Timestamps**: `CHANNEL_MAPPING_DF` for channel mapping, and `TONE_TIMESTAMP_DF` for tone timestamps, loaded from external sources.
- **Constants for DataFrame Columns**: Names for various columns in the DataFrame, defined in an all-caps snake case format, such as `EPHYS_INDEX_COL`, `LFP_INDEX_COL`, etc.

### Output & Utility
- **Processed Data**: The notebook outputs processed data, particularly the Z-scored LFP traces, which are critical for further analysis in neuroscience research.
- **Data Files**: Outputs are saved in various formats (`CSV`, `Pickle`) in a specified output directory (`OUTPUT_DIR`).
- **Visualization**: While not explicitly mentioned, the notebook has the potential for data visualization (plots) based on processed LFP data.

### Processing Workflow
1. **LFP Extraction and Preprocessing**: 
    - Iterates through recording sessions to process `.rec` files.
    - Applies a series of preprocessing steps like bandpass filtering, notch filtering, resampling, and Z-scoring on the LFP data.
    - Exception handling for cases where the recording doesn't contain specified stream IDs.

2. **DataFrame Manipulation and Merging**:
    - Filtering `TONE_TIMESTAMP_DF` for trials with obtained LFP.
    - Addition of trial numbers and merging with `CHANNEL_MAPPING_DF`.
    - Dropping unnecessary columns and restructuring for analysis.

3. **LFP Trace Extraction for Each Trial and Brain Region**: 
    - Linking LFP calculations with trials.
    - Creating new rows for each brain region, extracting baseline, trial, and combined LFP traces.
    - Results in a comprehensive DataFrame that combines trial information with corresponding LFP traces.

4. **Data Storage**:
    - Saving processed DataFrame in both `CSV` and `Pickle` formats for easy access and future use.

### Usage Notes
- The notebook is project-specific and tailored for a particular dataset structure, requiring modifications for different data formats.
- Users should ensure file paths and directory names match their project's structure and adjust constants and parameters as needed for their specific analysis requirements.
- The notebook forms a part of a larger research framework, thus necessitating compatibility checks with other components of the project.

### Dependencies
- Python Libraries: `sys`, `os`, `glob`, `numpy`, `pandas`, `spikeinterface`
- External Data: Channel mapping and tone timestamp files, along with Spikegadgets `.rec` files.

### Customization and Scalability
- The notebook's modular design allows for easy adaptation to different datasets or extensions to include additional processing steps.
- Functions and processing steps are clearly demarcated, facilitating straightforward updates or enhancements.

### Conclusion
This notebook is a vital tool in the preprocessing and analysis of LFP data from Spikegadgets recordings, integral to neuroscience research focused on social competition trials. It offers a structured approach to handle, process, and store electrophysiological data, ensuring reproducibility and efficiency in research workflows.

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
LFP_RESAMPLE_RATIO = EPHYS_SAMPLING_RATE / LFP_SAMPLING_RATE
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [9]:
EPHYS_INDEX_COL = "time_stamp_index"
LFP_INDEX_COL = "lfp_index"
EPHYS_TIMESTAMP_COL = "time"
RECORDING_FILE_COL = "recording_file"
RECORDING_DIR_COL = "recording_dir"
BASELINE_LFP_INDEX_RANGE_COL = "baseline_lfp_index_range"
TRIAL_LFP_INDEX_RANGE_COL = "trial_lfp_index_range"
BASELINE_EPHYS_INDEX_RANGE_COL = "baseline_ephys_index_range"
TRIAL_EPHYS_INDEX_RANGE_COL = "trial_ephys_index_range"
BASELINE_VIDEOFRAME_RANGE_COL = "baseline_videoframe_range"
TRIAL_VIDEOFRAME_RANGE_COL = "trial_videoframe_range"
CURRENT_SUBJECT_COL = "current_subject"
ALL_CH_LFP_COL = "all_ch_lfp"
SUBJECT_COL = "Subject"
TRIAL_NUMBER_COL = "trial_number"
SPIKE_INTERFACE_COL = "spike_interface"
EIB_COL = "eib"

In [10]:
LFP_TRACE_COLUMNS = ["session_dir", "recording", "metadata_dir", "metadata_file", "first_dtype_name", "first_item_data", "last_dtype_name", "last_item_data", 'all_subjects', 'current_subject', 'filename']
VIDEO_COLUMNS = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
      'session_path', 'first_dtype_name', 'first_item_data',
       'all_subjects', 'current_subject', 'filename']

In [11]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel(os.path.join(git_root, "data/channel_mapping.xlsx"))
# Spreadsheet of tone time
SPIKEGADGETS_EXTRACTED_DF = pd.read_pickle("./proc/rce_pilot_2_trodes_metadata.pkl")

In [12]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [13]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_*/*.rec")

In [14]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/standard/2023_06_14/20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_13/20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_12/20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_12/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_16/20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [16]:
FULL_LFP_TRACES_PKL = "{}_full_lfp_traces_and_video_frames.pkl".format(OUTPUT_PREFIX)

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [17]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Extracting the LFP

In [18]:
SPIKEGADGETS_EXTRACTED_DF

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ..."
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ..."
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ..."
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[1.1, 1.4]",7454593,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 4156, 5541, 6927, 831...","[[918755, 1118758], [3318755, 3518757], [56187...","[[49358, 67558], [70155, 79355], [360955, 3671..."
5,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.4,"[1.1, 1.4]",7454593,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 4156, 5541, 6927, 831...","[[918755, 1118758], [3318755, 3518757], [56187...","[[49358, 67558], [70155, 79355], [360955, 3671..."
6,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,"[[855, 1055], [3251, 3450], [5546, 5747], [734...","[[21, 27], [111, 126], [278, 321], [339, 393],...",20230614_114041_standard_comp_to_training_D3_s...,/scr

In [19]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)

            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)            
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
        
    




An exception occurred: stream_id trodes is not in ['ECU']
20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged
20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged
20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged
20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged
20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged
An exception occurred: stream_id trodes is not in ['ECU']
20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged
20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_bo

- Combining LFP traces with metadata

In [20]:
# Filtering for rows that are the metadata for the raw recordings
# lfp_trace_condition = (SPIKEGADGETS_EXTRACTED_DF["recording"].isin(recording_name_to_all_ch_lfp) & (SPIKEGADGETS_EXTRACTED_DF["metadata_dir"] == "raw") & (SPIKEGADGETS_EXTRACTED_DF["metadata_file"] == "timestamps"))
lfp_trace_condition = (SPIKEGADGETS_EXTRACTED_DF["recording"].isin(recording_name_to_all_ch_lfp))

SPIKEGADGETS_LFP_DF = SPIKEGADGETS_EXTRACTED_DF[lfp_trace_condition].copy().reset_index(drop=True)
# Removing the columns that are not needed
# SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF[LFP_TRACE_COLUMNS]

In [21]:
# Adding the LFP traces to the metadata dataframe
SPIKEGADGETS_LFP_DF["all_ch_lfp"] = SPIKEGADGETS_LFP_DF["recording"].map(recording_name_to_all_ch_lfp)

## Get the timestamp of the LFP

In [22]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,all_ch_lfp
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[1.1, 1.4]",7454593,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 4156, 5541, 6927, 831...","[[918755, 1118758], [3318755, 3518757], [56187...","[[49358, 67558], [70155, 79355], [360955, 3671...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...


In [23]:
# Getting the timestamps for the LFP traces by resampling the raw timestamps
SPIKEGADGETS_LFP_DF["LFP_timestamps"] = SPIKEGADGETS_LFP_DF.apply(lambda row: np.arange(row["raw_timestamps"][0], row["raw_timestamps"][0]+(row["all_ch_lfp"].get_total_samples()*LFP_RESAMPLE_RATIO), LFP_RESAMPLE_RATIO).astype(int), axis=1)
                                                                  
                                                                  


In [24]:
SPIKEGADGETS_LFP_DF

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,all_ch_lfp,LFP_timestamps
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[1.1, 1.4]",7454593,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 4156, 5541, 6927, 831...","[[918755, 1118758], [3318755, 3518757], [56187...","[[49358, 67558], [70155, 79355], [360955, 3671...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
5,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,2023061

In [25]:
SPIKEGADGETS_LFP_DF["all_ch_lfp"].iloc[0]

ZScoreRecording: 32 channels - 1.0kHz - 1 segments - 3,414,668 samples - 3,414.67s (56.91 minutes) 
                 float32 dtype - 416.83 MiB

In [26]:
SPIKEGADGETS_LFP_DF["LFP_timestamps"].iloc[0][0]


0

In [27]:
SPIKEGADGETS_LFP_DF["LFP_timestamps"].iloc[0][-1]

68293340

## Adding the channel mapping

In [28]:
CHANNEL_MAPPING_DF = CHANNEL_MAPPING_DF.drop(columns=[col for col in CHANNEL_MAPPING_DF.columns if "eib" in col], errors="ignore")

In [29]:
for col in CHANNEL_MAPPING_DF.columns:
    if "spike_interface" in col:
        CHANNEL_MAPPING_DF[col] = CHANNEL_MAPPING_DF[col].fillna(0)
        CHANNEL_MAPPING_DF[col] = CHANNEL_MAPPING_DF[col].astype(int).astype(str)

In [30]:
CHANNEL_MAPPING_DF

,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,21,15,14,13,16
1,1,6.2,0,0,0,0,0
2,1,6.3,0,0,0,0,0
3,1,6.4,0,0,0,0,0
4,2,1.1,5,31,30,29,28
5,2,1.2,10,31,30,29,28
6,2,1.3,9,31,30,29,28
7,2,1.4,15,31,30,29,28


- Adding all the brain region to ch information

In [31]:
CHANNEL_MAPPING_DF[SUBJECT_COL] = CHANNEL_MAPPING_DF[SUBJECT_COL].astype(str)

- Merging the recording and the channel dataframes

In [32]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,raw_timestamps,video_timestamps,tone_timestamps,port_entry_timestamps,all_ch_lfp,LFP_timestamps
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[1.3, 1.4]",8798886,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[1.1, 1.2]",7977066,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[1126742, 1326741], [3526740, 3726740], [5826...","[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[1.1, 1.4]",7454593,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1384, 1384, 2770, 4156, 4156, 5541, 6927, 831...","[[918755, 1118758], [3318755, 3518757], [56187...","[[49358, 67558], [70155, 79355], [360955, 3671...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."


In [33]:
SPIKEGADGETS_LFP_DF = pd.merge(SPIKEGADGETS_LFP_DF, CHANNEL_MAPPING_DF, left_on=CURRENT_SUBJECT_COL, right_on=SUBJECT_COL, how="left")



In [34]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,...,port_entry_timestamps,all_ch_lfp,LFP_timestamps,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[1.3, 1.4]",8798886,...,"[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.3,9,31,30,29,28
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[1.3, 1.4]",8798886,...,"[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.4,15,31,30,29,28
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1.1, 1.2]",7977066,...,"[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[1.1, 1.2]",7977066,...,"[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.2,10,31,30,29,28
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[1.1, 1.4]",7454593,...,"[[49358, 67558], [70155, 79355], [360955, 3671...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28


# Getting the channel specific LFP traces

- Linking up all LFP calculations with all the trials

- Extracting the traces for each brain region

In [35]:
SPIKEGADGETS_LFP_DF.head()

,session_dir,original_file,tone_frames,port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,...,port_entry_timestamps,all_ch_lfp,LFP_timestamps,Cohort,Subject,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[1.3, 1.4]",8798886,...,"[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.3,9,31,30,29,28
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[1.3, 1.4]",8798886,...,"[[491029, 515227], [519426, 558629], [559427, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.4,15,31,30,29,28
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1.1, 1.2]",7977066,...,"[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[1.1, 1.2]",7977066,...,"[[192745, 249350], [389747, 407142], [917544, ...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.2,10,31,30,29,28
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[1.1, 1.4]",7454593,...,"[[49358, 67558], [70155, 79355], [360955, 3671...",ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...",2,1.1,5,31,30,29,28


In [36]:
for col in SPIKEGADGETS_LFP_DF.columns:
    if "spike_interface" in col:
        print(col)
        brain_region = col.strip(SPIKE_INTERFACE_COL).strip("_")
        trace_column = "{}_lfp_trace".format(brain_region)
        SPIKEGADGETS_LFP_DF[trace_column] = SPIKEGADGETS_LFP_DF.apply(lambda row: row[ALL_CH_LFP_COL].get_traces(channel_ids=[row[col]]).T[0], axis=1)
                                                                                                                                                        
                                                                                                                                                    

spike_interface_mPFC


## Reorganizing the dataframe and removing unnecessary columns

In [ ]:
SPIKEGADGETS_LFP_DF.columns

In [ ]:
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF.drop(columns=[col for col in SPIKEGADGETS_LFP_DF.columns if "spike_interface" in col], errors="ignore")
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF.drop(columns=['all_ch_lfp', 'original_file', 'filename', 'current_subject'], errors="ignore")
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF.rename(columns={col: col.lower() for col in SPIKEGADGETS_LFP_DF.columns})

In [ ]:
sorted_columns = sorted(SPIKEGADGETS_LFP_DF.columns
, key=lambda x: x.split("_")[-1])

In [ ]:
SPIKEGADGETS_LFP_DF = SPIKEGADGETS_LFP_DF[sorted_columns]

In [ ]:
SPIKEGADGETS_LFP_DF.head()

In [ ]:
SPIKEGADGETS_LFP_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
SPIKEGADGETS_LFP_DF["recording"].unique()

In [ ]:
raise ValueError("Stop here")